<a href="https://colab.research.google.com/github/mle-invn/mobilenetv3-tensorflow/blob/master/cifar10_mobilenetv3_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
!git clone https://github.com/Bisonai/mobilenetv3-tensorflow.git
%cd mobilenetv3-tensorflow 

Cloning into 'mobilenetv3-tensorflow'...
remote: Enumerating objects: 67, done.
remote: Total 67 (delta 0), reused 0 (delta 0), pack-reused 67
Unpacking objects: 100% (67/67), done.
/content/mobilenetv3-tensorflow/mobilenetv3-tensorflow


In [61]:
import tensorflow as tf
import tensorflow_datasets as tfds
from mobilenetv3_factory import build_mobilenetv3
from datasets import build_dataset

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

In [54]:
# Parameters
name = "cifar10"
train_batch_size = 128
valid_batch_size = 128
height = 128
width = 128
model_type = "small"
l2_reg = 0.0001
lr = 0.01
width_multiplier = 1.2
#optimizer = tf.keras.optimizers.RMSprop(lr)
optimizer = tf.keras.optimizers.Adam(lr)
num_epoch = 100

In [67]:
def _parse_function(data, shape, num_classes, channels):
    height, width = shape
    image = data["image"]
    label = data["label"]

    image = tf.cast(image, dtype=tf.float32)
    image = tf.image.resize(image, (height,width))
    image = tf.reshape(image, (height,width, channels))
    image = image / 255.0
    image = image - 0.5
    image = image * 2.0
    label = tf.one_hot(label, depth=num_classes)

    return image, label
shape = [height,width]

dataset = {}
builder = tfds.builder(name)
dataset["num_train"] = builder.info.splits['train'].num_examples
dataset["num_test"] = builder.info.splits['test'].num_examples

[ds_train, ds_test], info = tfds.load(name=name, split=["train", "test"], with_info=True)
dataset["num_classes"] = info.features["label"].num_classes
dataset["channels"] = info.features['image'].shape[-1]

ds_train = ds_train.shuffle(1024).repeat()
ds_train = ds_train.map(lambda data: _parse_function(data, [height,width], dataset["num_classes"], dataset["channels"]))
dataset["train"] = ds_train.batch(train_batch_size)

ds_test = ds_test.shuffle(1024).repeat()
ds_test = ds_test.map(lambda data: _parse_function(data, shape, dataset["num_classes"], dataset["channels"]))
dataset["test"] = ds_test.batch(valid_batch_size)

In [55]:
dataset = build_dataset(
        name=name,
        shape=[height,width],
        train_batch_size=train_batch_size,
        valid_batch_size=valid_batch_size
        )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/cifar10/3.0.2.incomplete72YUBU/cifar10-train.tfrecord*...:   0%|          | 0/…

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/cifar10/3.0.2.incomplete72YUBU/cifar10-test.tfrecord*...:   0%|          | 0/1…

Dataset cifar10 downloaded and prepared to ~/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


AttributeError: ignored

In [68]:
model = build_mobilenetv3(
        model_type,
        input_shape=(height, width, dataset["channels"]),
        num_classes=dataset["num_classes"],
        width_multiplier=width_multiplier,
        l2_reg=l2_reg,
    )

In [69]:
model.compile(
        optimizer= optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"],)

In [ ]:
model.fit(
        tf.compat.v1.data.make_one_shot_iterator(dataset["train"]),
        # dataset["train"].make_one_shot_iterator(),
        steps_per_epoch=(dataset["num_train"]//train_batch_size)+1,
        epochs=num_epoch,
        validation_data=dataset["test"],
        validation_steps=(dataset["num_test"]//valid_batch_size)+1,
    )

Epoch 1/100
391/391 [==============================] - 70s 136ms/step - loss: 2.7496 - accuracy: 0.1407 - val_loss: 10.9319 - val_accuracy: 0.0974
Epoch 2/100
391/391 [==============================] - 51s 130ms/step - loss: 2.4406 - accuracy: 0.1690 - val_loss: 2.2182 - val_accuracy: 0.2266
Epoch 3/100
391/391 [==============================] - 51s 131ms/step - loss: 1.9886 - accuracy: 0.2928 - val_loss: 1.9216 - val_accuracy: 0.3099
Epoch 4/100
391/391 [==============================] - 51s 131ms/step - loss: 1.7664 - accuracy: 0.3828 - val_loss: 1.8997 - val_accuracy: 0.3506
Epoch 5/100
391/391 [==============================] - 52s 133ms/step - loss: 1.6308 - accuracy: 0.4352 - val_loss: 1.5492 - val_accuracy: 0.4812
Epoch 6/100
391/391 [==============================] - 52s 133ms/step - loss: 1.4947 - accuracy: 0.4908 - val_loss: 1.7965 - val_accuracy: 0.4016
Epoch 7/100
391/391 [==============================] - 51s 131ms/step - loss: 1.3735 - accuracy: 0.5439 - val_loss: 1.5726 

In [ ]:
print(tf.config.list_physical_devices('GPU'))